## pitchとplayerの前処理を統合
#### 15
- all_pitch_10

#### 14
- all_pitch_9 ,all_player_11, all_pitcher_player_1.f

#### 13
- all_pitch_8 ,all_player_10

In [1]:
import pandas as pd
import feather
pd.set_option('display.max_Columns', 300)

In [2]:
ALL_PITCH = 'intermediate/all_pitch_10.f'
ALL_PITCHER = 'intermediate/all_pitcher_player_1.f'
ALL_PLAYER = 'intermediate/all_player_11.f'
OUTPUT = 'intermediate/all_merge_15.f'

In [3]:
all_pitch = pd.read_feather(ALL_PITCH)
all_pitch.shape

(778767, 92)

In [4]:
all_pitcher = pd.read_feather(ALL_PITCHER)
all_pitcher.shape

(2832, 28)

In [5]:
all_player = pd.read_feather(ALL_PLAYER)
all_player.shape

(2757, 18)

In [6]:
all_pitch.head()

,データ内連番,ball,投球位置区域,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:00,00:00:00,0.000000,NaN,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0.0,8.0,2017,2017033101,2,2,1,4,1,1,2,1500001,1,1,1,2,2,900410,4,1,1,0,0,0,0,1,___,1300027,R_L,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:12,00:00:12,0.200000,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,0.0,5.0,2017,2017033101,3,3,1,4,1,1,3,1500001,1,1,1,3,3,900410,4,1,1,0,0,0,0,2,___,1300027,R_L,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:23,00:00:23,0.383333,0.183333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3,0.0,12.0,2017,2017033101,4,4,1,4,1,2,1,1500001,1,1,2,4,4,11436,4,2,1,0,0,1,0,0,___,1300027,R_R,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:04:01,00:01:01,1.016667,0.633333,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,0.0,8.0,2017,2017033101,5,5,1,4,1,2,2,1500001,1,1,2,5,5,11436,4,2,1,0,0,1,0,1,___,1300027,R_R,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:04:12,00:01:12,1.200000,0.183333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Join
- pitch.投手ID/年度 - player.選手ID/年度
- pitch.打者ID/年度 - player.選手ID/年度
- pitch.捕手ID/年度 - player.選手ID/年度

In [7]:
merge_all = pd.merge(all_pitch, all_pitcher, left_on=['投手ID', '年度', 'pit_bat'], right_on=['選手ID', '年度', 'pit_bat'], how='left')

In [8]:
merge_all = pd.merge(merge_all, all_player, left_on=['打者ID', '年度'], right_on=['選手ID', '年度'], how='left', suffixes=['_pit', '_bat'])

In [9]:
merge_all = pd.merge(merge_all, all_player.drop(columns=['batter_cnt', 'bat_game_cnt']), left_on=['捕手ID', '年度'], right_on=['選手ID', '年度'], how='left')

### player同士の組み合わせ
- 年棒の差
- 現役年数の差
- 年齢の差
- salary_yearの差
- salary_x_yearの差
- BMIの差

In [10]:
merge_all.head()

,データ内連番,ball,投球位置区域,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,選手ID_pit,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,選手ID_bat,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:00,00:00:00,0.000000,NaN,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,900410,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,1300027,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187
1,1,0.0,8.0,2017,2017033101,2,2,1,4,1,1,2,1500001,1,1,1,2,2,900410,4,1,1,0,0,0,0,1,___,1300027,R_L,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:12,00:00:12,0.200000,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,900410,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,1300027,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187
2,2,0.0,5.0,2017,2017033101,3,3,1,4,1,1,3,1500001,1,1,1,3,3,900410,4,1,1,0,0,0,0,2,___,1300027,R_L,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 18:03:00,2020-05-09 18:03:23,00:00:23,0.383333,0.183333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,900410,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,1300027,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187
3,3,0.0,12.0,2017,2017033101,4,4,1,4,1,2,1,1500001,1,1,2,4,4,11436,4,2,1,0,0,1,0,0,___,1300027,R_R,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,1,1,0,2017-03-31,2017-03-31,0,2020-05-09 

In [11]:
merge_all['salary_dif_p-b'] = merge_all['salary_pit'] - merge_all['salary_bat']
merge_all['play_year_dif_p-b'] = merge_all['play_year_pit'] - merge_all['play_year_bat']
merge_all['age_dif_p-b'] = merge_all['age_pit'] - merge_all['age_bat']
merge_all['salary_year_dif_p-b'] = merge_all['salary_year_pit'] - merge_all['salary_year_bat']
merge_all['salary_x_year_dif_p-b'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year_bat']
merge_all['bmi_dif_p-b'] = merge_all['bmi_pit'] - merge_all['bmi_bat']

merge_all['salary_dif_p-c'] = merge_all['salary_pit'] - merge_all['salary']
merge_all['play_year_dif_p-c'] = merge_all['play_year_pit'] - merge_all['play_year']
merge_all['age_dif_p-c'] = merge_all['age_pit'] - merge_all['age']
merge_all['salary_year_dif_p-c'] = merge_all['salary_year_pit'] - merge_all['salary_year']
merge_all['salary_x_year_dif_p-c'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year']
merge_all['bmi_dif_p-c'] = merge_all['bmi_pit'] - merge_all['bmi']

merge_all['salary_dif_b-c'] = merge_all['salary_bat'] - merge_all['salary']
merge_all['play_year_dif_b-c'] = merge_all['play_year_bat'] - merge_all['play_year']
merge_all['age_dif_b-c'] = merge_all['age_bat'] - merge_all['age']
merge_all['salary_year_dif_b-c'] = merge_all['salary_year_bat'] - merge_all['salary_year']
merge_all['salary_x_year_dif_b-c'] = merge_all['salary_x_year_bat'] - merge_all['salary_x_year']
merge_all['bmi_dif_b-c'] = merge_all['bmi_bat'] - merge_all['bmi']

In [12]:
merge_all = pd.get_dummies(merge_all, columns=['pit_bat'])

### 不要な列を削除

In [13]:
merge_all.drop(columns=[
    '選手ID_pit', '選手ID_bat', '選手ID',
    '年度', 
    '試合ID', 
    'ホームチームID', 'アウェイチームID', 
    '投手ID', '投手チームID', 
    '打者ID', '打者チームID', 
    'プレイ前走者状況', 
    '捕手ID', 
    'opening_date', 'game_date',
    'start_time', 'game_time', 'elapsed_time'
], inplace=True)

### Rename

In [14]:
merge_all.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [15]:
print(merge_all.shape)
merge_all.head()

(778767, 153)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,0,0.000000,NaN,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,0,0.200000,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,0,0.383333,0.183333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0,176,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.06

In [16]:
merge_all.to_feather(OUTPUT)

### GCS
#### local->GCS

In [17]:
GCS = 'gs://yasunobu_baseball/'

In [18]:
'gsutil cp ./{} {}intermediate/'.format(OUTPUT, GCS)

'gsutil cp ./intermediate/all_merge_15.f gs://yasunobu_baseball/intermediate/'

#### GCS->GCE

In [19]:
'gsutil cp {}{} ~/work/baseball/intermediate'.format(GCS, OUTPUT)

'gsutil cp gs://yasunobu_baseball/intermediate/all_merge_15.f ~/work/baseball/intermediate'